In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("drive/MyDrive/Colab Notebooks/movies_subtitles.csv", engine="python")
data.head()

,start_time,end_time,text,imdb_id
0,58.559,61.602,"BOY: All right, everyone!\nThis... is a stick-up!",tt0114709
1,61.687,63.354,Don't anybody move!,tt0114709
2,64.398,66.482,"Now, empty that safe!",tt0114709
3,68.318,71.612,"Ooh-hoo-hoo!\nMoney, money, money! (KISSING)",tt0114709
4,71.697,74.031,"Stop it! Stop it,\nyou mean, old potato!",tt0114709


После попытки обучения модели на датасете и появлении ошибки я догадался, что с данными что-то не так. Оказалось, что в данных целых 27 строк с незаполненным текстом. Выведу первые пять таких строк.

In [ ]:
data[data["text"].isna()].head()

,start_time,end_time,text,imdb_id
7452,6898.0,6901.134,NaN,tt0112637
32877,5495.0,5498.084,NaN,tt0112495
41861,5087.0,5090.095,NaN,tt0112508
83269,7205.0,7208.084,NaN,tt0111742
102811,5603.0,5606.136,NaN,tt0107756


Вообще удалим такие строки, так как мы не знаем, чем их лучше всего заполнить

In [ ]:
data = data.drop(data[data["text"].isna()].index)
data[data['text'].isna()]

,start_time,end_time,text,imdb_id


In [ ]:
def get_text(x):
    return x["text"]

In [ ]:
texts = np.array(get_text(data.loc[:1000000]))
texts = np.array(texts)
(texts)
max_tokens = 5000
texts

<class 'str'>


array(['BOY: All right, everyone! This... is a stick-up!',
       "Don't anybody move!", 'Now, empty that safe!', ...,
       "If she stumbled on a burglary, she should've been here.",
       'But she was killed exactly where she is.',
       'See, the blood patterns indicate that.'], dtype=object)

In [ ]:
def transform(x):
   x = str(x)
   x = x.replace('\n', ' ')
   x = x.encode("utf8").decode("ascii", 'ignore')
   return x

In [ ]:
data["word_count"] = data["text"].apply(lambda x: len(str(x).split()))
data["text"] = data["text"].apply(transform)
data.head()

,start_time,end_time,text,imdb_id,word_count
0,58.559,61.602,"BOY: All right, everyone! This... is a stick-up!",tt0114709,8
1,61.687,63.354,Don't anybody move!,tt0114709,3
2,64.398,66.482,"Now, empty that safe!",tt0114709,4
3,68.318,71.612,"Ooh-hoo-hoo! Money, money, money! (KISSING)",tt0114709,5
4,71.697,74.031,"Stop it! Stop it, you mean, old potato!",tt0114709,8


In [ ]:
max_len = data["word_count"].max()
max_len

3123

In [ ]:
max_tokens = 5000
vocab_size = 5000
batch_size = 128
embed_size = 64
samples_size = 1000000

In [ ]:
vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_tokens, input_shape=(1,))
vectorizer.adapt(texts)

In [ ]:
(vectorizer.get_vocabulary())

['',
 '[UNK]',
 'you',
 'the',
 'i',
 'to',
 'a',
 'and',
 'it',
 'of',
 'that',
 'in',
 'me',
 'is',
 'what',
 'this',
 'on',
 'for',
 'my',
 'your',
 'do',
 'no',
 'dont',
 'we',
 'have',
 'im',
 'be',
 'know',
 'he',
 'all',
 'its',
 'not',
 'are',
 'was',
 'with',
 'get',
 'here',
 'just',
 'right',
 'like',
 'but',
 'go',
 'come',
 'so',
 'out',
 'up',
 'him',
 'youre',
 'got',
 'well',
 'now',
 'about',
 'can',
 'there',
 'if',
 'at',
 'oh',
 'thats',
 'they',
 'were',
 'one',
 'yeah',
 'see',
 'want',
 'how',
 'good',
 'think',
 'her',
 'will',
 'did',
 'his',
 'she',
 'as',
 'look',
 'yes',
 'ill',
 'why',
 'from',
 'hes',
 'gonna',
 'going',
 'man',
 'take',
 'back',
 'when',
 'time',
 'okay',
 'them',
 'who',
 'an',
 'cant',
 'l',
 'tell',
 'us',
 'say',
 'some',
 'been',
 'down',
 'or',
 'hey',
 'would',
 'had',
 'where',
 'never',
 'could',
 'little',
 'way',
 'very',
 'didnt',
 'then',
 'whats',
 'too',
 'over',
 'mr',
 'something',
 'by',
 'more',
 'make',
 'our',
 'let',

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(None, 5000)),
    tf.keras.layers.Masking(mask_value=0),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation="softmax")
])
model.build((None, None, 5000))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, None, 5000)        0         
                                                                 
 lstm (LSTM)                 (None, None, 128)         2626048   
                                                                 
 dense (Dense)               (None, None, 5000)        645000    
                                                                 
Total params: 3,271,048
Trainable params: 3,271,048
Non-trainable params: 0
_________________________________________________________________


In [ ]:
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(19, embed_size),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(vocab_size, activation="softmax")
])
lstm_model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, None, 64)          1216      
                                                                 
 lstm_25 (LSTM)              (None, 128)               98816     
                                                                 
 dense_15 (Dense)            (None, 5000)              645000    
                                                                 
Total params: 745,032
Trainable params: 745,032
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def prepare_data(x):
  x = vectorizer(x)
  y = x[:, 1:]
  col = tf.constant(2, shape=(x.shape[0],1), dtype="int64")
  y = tf.concat([y, col], 1)
  out1 = tf.one_hot(x, vocab_size)
  out2 = tf.one_hot(y, vocab_size)
  return out1, out2

In [ ]:
def prepare(x):
  x = vectorizer(x)
  y = x[:, 1:]
  col = tf.constant(0, shape=(x.shape[0],1), dtype="int64")
  y = tf.concat([y, col], 1)
  return x, y

In [ ]:
prepare(["hello world", "nice to meet you"])

(<tf.Tensor: shape=(2, 4), dtype=int64, numpy=
 array([[212, 269,   0,   0],
        [204,   5, 333,   2]])>,
 <tf.Tensor: shape=(2, 4), dtype=int64, numpy=
 array([[269,   0,   0,   0],
        [  5, 333,   2,   0]])>)

Протестируем функцию

In [ ]:
x = ["hello world", "nice to meet you"]
prepare_data(x)

(<tf.Tensor: shape=(2, 4, 5000), dtype=float32, numpy=
 array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>,
 <tf.Tensor: shape=(2, 4, 5000), dtype=float32, numpy=
 array([[[0., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.]]], dtype=float32)>)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0006), loss='categorical_crossentropy',metrics=['acc'])

index=0
X, Y = prepare_data(np.array(get_text(data.loc[index * batch_size:index * batch_size + batch_size])))
model.fit(X, Y,epochs=200)


Epoch 1/200
5/5 [==============================] - 7s 250ms/step - loss: 0.0785 - acc: 0.9664
Epoch 2/200
5/5 [==============================] - 1s 241ms/step - loss: 0.0778 - acc: 0.9680
Epoch 3/200
5/5 [==============================] - 1s 226ms/step - loss: 0.0769 - acc: 0.9685
Epoch 4/200
5/5 [==============================] - 1s 241ms/step - loss: 0.0762 - acc: 0.9690
Epoch 5/200
5/5 [==============================] - 1s 268ms/step - loss: 0.0754 - acc: 0.9690
Epoch 6/200
5/5 [==============================] - 2s 404ms/step - loss: 0.0755 - acc: 0.9695
Epoch 7/200
5/5 [==============================] - 2s 408ms/step - loss: 0.0756 - acc: 0.9695
Epoch 8/200
5/5 [==============================] - 1s 245ms/step - loss: 0.0754 - acc: 0.9695
Epoch 9/200
5/5 [==============================] - 1s 243ms/step - loss: 0.0754 - acc: 0.9685
Epoch 10/200
5/5 [==============================] - 1s 235ms/step - loss: 0.0753 - acc: 0.9685
Epoch 11/200
5/5 [==============================] - 1s 241m

Сгенерируем текст с помощью нашей модели

In [ ]:
def generate(model, size, sentence):
  start = sentence
  for i in range(0, size):
    v = vectorizer([start])[0]
    out1 = tf.one_hot(v, vocab_size)
    out1 = tf.reshape(out1, [1, out1.shape[0], out1.shape[1]])
    logits = model(out1)[0][-1]
    nc = tf.argmax(logits)
    print(vectorizer.get_vocabulary()[nc], end= " ")
    if vectorizer.get_vocabulary()[nc] == "eos" or vectorizer.get_vocabulary()[nc] == "": break
    start = vectorizer.get_vocabulary()[nc]

In [ ]:
def generate_soft(model, size, sentence, temperature=1.0):
   start = sentence
   for i in range(0, size):
     v = vectorizer([start])[0]
     out1 = tf.one_hot(v, vocab_size)
     out1 = tf.reshape(out1, [1, out1.shape[0], out1.shape[1]])
     logits = model(out1)[0][-1]
     probs = tf.exp(tf.math.log(logits) / temperature).numpy().astype(np.float64)
     probs = probs / np.sum(probs)
     nc = np.argmax(np.random.multinomial(1, probs, 1))
     print(vectorizer.get_vocabulary()[nc], end= " ")
     if vectorizer.get_vocabulary()[nc] == "eos" or vectorizer.get_vocabulary()[nc] == "": break
     start = vectorizer.get_vocabulary()[nc]

In [ ]:
data.head(60)

,start_time,end_time,text,imdb_id,word_count
0,58.559,61.602,"BOY: All right, everyone! This... is a stick-up!",tt0114709,8
1,61.687,63.354,Don't anybody move!,tt0114709,3
2,64.398,66.482,"Now, empty that safe!",tt0114709,4
3,68.318,71.612,"Ooh-hoo-hoo! Money, money, money! (KISSING)",tt0114709,5
4,71.697,74.031,"Stop it! Stop it, you mean, old potato!",tt0114709,8
5,74.116,77.160,"Quiet, Bo Peep, or your sheep get run over!",tt0114709,9
6,77.327,79.328,Help! Baa! Help us!,tt0114709,4
7,79.413,82.957,"Oh, no, not my sheep! Somebody do something!",tt0114709,8
8,89.339,92.425,VOICE BOX: Reach for the sky!,tt0114709,6
9,92.509,94.677,"Oh, no! Sheriff Woody!",tt0114709,4


In [ ]:
generate(model, 20, ["here"])

woody [UNK]  

In [ ]:
start_words = ["empty", "boring", "happy", "bridge",  "you", "where", "when", "take", "us", "want", "remember", "friendship", "coming", "expected", "make", "do", "all", "birthday", "chance"]
temperature_rates = [1, 1.2, 1.4, 1.6]
for word in start_words:
  for temperature in temperature_rates:
    print("[START WORD]: ", word, " [TEMPERATURE]:", temperature)
    generate_soft(model, 20, [word], temperature=temperature)
    print()
    print()

[START WORD]:  empty  [TEMPERATURE]: 1
that move as the cousin breathe ouch [UNK] come on [UNK] come on oh youre going for crying tai [UNK] 

[START WORD]:  empty  [TEMPERATURE]: 1.2
first [UNK] come on wrap as [UNK] [UNK] box gather everyone for hey [UNK] come on come on box be 

[START WORD]:  empty  [TEMPERATURE]: 1.4
no [UNK] howd me dreadful tickets it you can andrews [UNK] you can be happy ha [UNK] box ishe for 

[START WORD]:  empty  [TEMPERATURE]: 1.6
teams amazing for forgive journey friends tree bottle brief hates kong chattering loving arliss box wagon glorious ernie donald mcmurphy 

[START WORD]:  boring  [TEMPERATURE]: 1
[UNK]  

[START WORD]:  boring  [TEMPERATURE]: 1.2
price [UNK]  

[START WORD]:  boring  [TEMPERATURE]: 1.4
youve got it promises happened [UNK]  

[START WORD]:  boring  [TEMPERATURE]: 1.6
serial no [UNK] money moved moms all right here woody hot charge field blows state bad roxanne 90 units gonna 

[START WORD]:  happy  [TEMPERATURE]: 1
ha got a go as [

Обучим многослойную STLM

In [ ]:
multilayer_stlm = tf.keras.Sequential([
    tf.keras.Input(shape=(None, 5000)),
    tf.keras.layers.Masking(mask_value=0),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation="softmax")
])
multilayer_stlm.build((None, None, 5000))
multilayer_stlm.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_4 (Masking)         (None, None, 5000)        0         
                                                                 
 lstm_12 (LSTM)              (None, None, 128)         2626048   
                                                                 
 lstm_13 (LSTM)              (None, None, 128)         131584    
                                                                 
 lstm_14 (LSTM)              (None, None, 128)         131584    
                                                                 
 dense_4 (Dense)             (None, None, 5000)        645000    
                                                                 
Total params: 3,534,216
Trainable params: 3,534,216
Non-trainable params: 0
_________________________________________________________________


In [ ]:
multilayer_stlm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005), loss='categorical_crossentropy',metrics=['acc'])

index_=0
X_, Y_ = prepare_data(np.array(get_text(data.loc[index * batch_size:index * batch_size + batch_size])))
multilayer_stlm.fit(X_, Y_,epochs=200)


Epoch 1/200
5/5 [==============================] - 16s 297ms/step - loss: 0.6124 - acc: 0.8532
Epoch 2/200
5/5 [==============================] - 2s 296ms/step - loss: 0.6116 - acc: 0.8537
Epoch 3/200
5/5 [==============================] - 2s 291ms/step - loss: 0.6113 - acc: 0.8553
Epoch 4/200
5/5 [==============================] - 2s 300ms/step - loss: 0.6114 - acc: 0.8548
Epoch 5/200
5/5 [==============================] - 1s 286ms/step - loss: 0.6110 - acc: 0.8537
Epoch 6/200
5/5 [==============================] - 2s 299ms/step - loss: 0.6111 - acc: 0.8532
Epoch 7/200
5/5 [==============================] - 2s 434ms/step - loss: 0.6105 - acc: 0.8537
Epoch 8/200
5/5 [==============================] - 3s 494ms/step - loss: 0.6104 - acc: 0.8543
Epoch 9/200
5/5 [==============================] - 2s 300ms/step - loss: 0.6102 - acc: 0.8548
Epoch 10/200
5/5 [==============================] - 2s 304ms/step - loss: 0.6101 - acc: 0.8543
Epoch 11/200
5/5 [==============================] - 2s 301

KeyboardInterrupt: ignored

In [ ]:
generate(multilayer_stlm, 20, ["think"])

In [ ]:
generate_soft(multilayer_stlm, 20, ["never"], temperature=1.4)

going remember im explosion bad anybody 3 its [UNK]  

In [ ]:
start_words = ["empty", "boring", "happy", "bridge"  "you", "where", "when", "take", "us", "want", "remember", "friendship", "coming", "expected", "make", "do", "all", "birthday", "chance"]
temperature_rates = [1, 1.2, 1.4, 1.6]
for word in start_words:
  for temperature in temperature_rates:
    print("[START WORD]: ", word, " [TEMPERATURE]:", temperature)
    generate_soft(multilayer_stlm, 20, [word], temperature=temperature)
    print()
    print()

[START WORD]:  empty  [TEMPERATURE]: 1
[UNK] whoa hey [UNK] [UNK] a have come [UNK] ow for [UNK] 3 running what hear what me [UNK]  

[START WORD]:  empty  [TEMPERATURE]: 1.2
gonna have box no ha sure everybody in box [UNK] bad [UNK] [UNK] come youre running im going now oh 

[START WORD]:  empty  [TEMPERATURE]: 1.4
[UNK]  

[START WORD]:  empty  [TEMPERATURE]: 1.6
we oh woody me be [UNK] face meeting it for lets bo shh  

[START WORD]:  boring  [TEMPERATURE]: 1
got want im going i wanted first im going remember im here anybody on [UNK] howd [UNK] youre going got 

[START WORD]:  boring  [TEMPERATURE]: 1.2
meeting got going remember what hear what i think  

[START WORD]:  boring  [TEMPERATURE]: 1.4
got where thank  

[START WORD]:  boring  [TEMPERATURE]: 1.6
be gonna toys [UNK] oh [UNK] for [UNK] up [UNK] right here  

[START WORD]:  happy  [TEMPERATURE]: 1
[UNK] the birthday [UNK] for on [UNK] it happy  

[START WORD]:  happy  [TEMPERATURE]: 1.2
right going meeting friendship  

[STA

**Модели с посимвольной токенизацией**

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts(list(get_text(data.loc[0:10000])))

In [ ]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1
maxlen = data.loc[:100000]["text"].apply(lambda x: len(x)).max()
maxlen

312

In [ ]:
symbol_simple_rnn = tf.keras.models.Sequential([
    tf.keras.layers.Masking(input_shape=(maxlen,vocab_size)),
    tf.keras.layers.SimpleRNN(128,return_sequences=True,input_shape=(maxlen,vocab_size)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size,activation='softmax'))
])
symbol_simple_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, 312, 83)           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 312, 128)          27136     
                                                                 
 time_distributed (TimeDistr  (None, 312, 83)          10707     
 ibuted)                                                         
                                                                 
Total params: 37,843
Trainable params: 37,843
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def prepare_for_symbol(x):
  x = tokenizer.texts_to_sequences(x)
  y = [i[1:] + [eos_token] for i in x]
  x = tf.keras.preprocessing.sequence.pad_sequences(x,padding='post',maxlen=maxlen)
  y = tf.keras.preprocessing.sequence.pad_sequences(y,padding='post',maxlen=maxlen)
  out1 = tf.one_hot(x, vocab_size)
  out2 = tf.one_hot(y, vocab_size)
  return out1, out2

В процессе обучения я уменьшаю темп обучения в 2 раза, если лосс перестаёт монотонно падать. По-хорошему для обучения стоило подобрать LRScheduler, который менят темп обучения по некоторому правилу.

In [ ]:
symbol_simple_rnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy',metrics=['acc'])

x1, y1 = prepare_for_symbol(list(get_text(data.loc[:1000])))
symbol_simple_rnn.fit(x1, y1, epochs=200)

Epoch 1/200
32/32 [==============================] - 5s 131ms/step - loss: 0.1063 - acc: 0.9696
Epoch 2/200
32/32 [==============================] - 5s 145ms/step - loss: 0.1040 - acc: 0.9708
Epoch 3/200
32/32 [==============================] - 4s 121ms/step - loss: 0.1033 - acc: 0.9712
Epoch 4/200
32/32 [==============================] - 4s 127ms/step - loss: 0.1030 - acc: 0.9713
Epoch 5/200
32/32 [==============================] - 5s 150ms/step - loss: 0.1027 - acc: 0.9714
Epoch 6/200
32/32 [==============================] - 4s 121ms/step - loss: 0.1025 - acc: 0.9716
Epoch 7/200
32/32 [==============================] - 4s 124ms/step - loss: 0.1024 - acc: 0.9715
Epoch 8/200
32/32 [==============================] - 5s 154ms/step - loss: 0.1026 - acc: 0.9715
Epoch 9/200
32/32 [==============================] - 4s 124ms/step - loss: 0.1024 - acc: 0.9716
Epoch 10/200
32/32 [==============================] - 4s 121ms/step - loss: 0.1022 - acc: 0.9718
Epoch 11/200
32/32 [===================

In [ ]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
  if x == 0: return " "
  return reverse_map[x]

In [ ]:
def generate_soft_for_symbol(model, size, sentence, temperature=1.0):
   start = sentence
   for i in range(0, size):
     ch = tokenizer.texts_to_sequences(start)[0]
     out1 = tf.one_hot(ch, vocab_size)
     out1 = tf.reshape(out1, shape=(1, out1.shape[0], out1.shape[1]))
     out1 = tf.keras.preprocessing.sequence.pad_sequences(out1,padding='post',maxlen=maxlen)
     logits = model(out1)[0][-1]
     probs = tf.exp(tf.math.log(logits) / temperature).numpy().astype(np.float64)
     probs = probs / np.sum(probs)
     nc = np.argmax(np.random.multinomial(1, probs, 1))
     print(decode(nc), end='')
     if nc == eos_token: break
     start = decode(nc)

In [ ]:
start_words = ["empty", "boring", "happy", "bridge", "you", "where", "when", "take", "us", "want", "remember", "friendship", "coming", "expected", "make", "do", "all", "birthday", "chance"]
temperature_rates = [1, 1.2, 1.4, 1.6]
for word in start_words:
  for temperature in temperature_rates:
    print("[START WORD]: ", word, " [TEMPERATURE]:", temperature)
    generate_soft_for_symbol(symbol_simple_rnn, 20, [word], temperature=temperature)
    print()
    print()

[START WORD]:  empty  [TEMPERATURE]: 1
inonouzzzzzzzzzzzzzz

[START WORD]:  empty  [TEMPERATURE]: 1.2
 Whencono.<eos>

[START WORD]:  empty  [TEMPERATURE]: 1.4
palouzzzzzzzzzzzzzza

[START WORD]:  empty  [TEMPERATURE]: 1.6
w!<eos>

[START WORD]:  boring  [TEMPERATURE]: 1
inon'sered ANonere's

[START WORD]:  boring  [TEMPERATURE]: 1.2
oinonelere Whedous S

[START WORD]:  boring  [TEMPERATURE]: 1.4
everevererell"<eos>

[START WORD]:  boring  [TEMPERATURE]: 1.6
erononelllsed ecal's

[START WORD]:  happy  [TEMPERATURE]: 1
 Wouthanowh, Tanexav

[START WORD]:  happy  [TEMPERATURE]: 1.2
 Covenoredinekizzzzz

[START WORD]:  happy  [TEMPERATURE]: 1.4
 Dofol.<eos>

[START WORD]:  happy  [TEMPERATURE]: 1.6
!<eos>

[START WORD]:  bridge  [TEMPERATURE]: 1
se!<eos>

[START WORD]:  bridge  [TEMPERATURE]: 1.2
s Bun'sered Wh, WO-(

[START WORD]:  bridge  [TEMPERATURE]: 1.4
llounofreremevereven

[START WORD]:  bridge  [TEMPERATURE]: 1.6
s WOfivepavedy methy

[START WORD]:  you  [TEMPERATURE]: 1
 Siveviv

Однослойная LSTM

In [ ]:
symbol_lstm = tf.keras.models.Sequential([
    tf.keras.layers.Masking(input_shape=(maxlen,vocab_size)),
    tf.keras.layers.LSTM(128,return_sequences=True,input_shape=(maxlen,vocab_size)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size,activation='softmax'))
])
symbol_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 312, 83)           0         
                                                                 
 lstm (LSTM)                 (None, 312, 128)          108544    
                                                                 
 time_distributed_1 (TimeDis  (None, 312, 83)          10707     
 tributed)                                                       
                                                                 
Total params: 119,251
Trainable params: 119,251
Non-trainable params: 0
_________________________________________________________________


In [ ]:
symbol_lstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='categorical_crossentropy',metrics=['acc'])

symbol_lstm.fit(x1, y1, epochs=200)

Epoch 1/200
32/32 [==============================] - 16s 387ms/step - loss: 0.9441 - acc: 0.8540
Epoch 2/200
32/32 [==============================] - 13s 390ms/step - loss: 0.5833 - acc: 0.8817
Epoch 3/200
32/32 [==============================] - 13s 395ms/step - loss: 0.7296 - acc: 0.8521
Epoch 4/200
32/32 [==============================] - 13s 395ms/step - loss: 0.4149 - acc: 0.9000
Epoch 5/200
32/32 [==============================] - 13s 393ms/step - loss: 0.3921 - acc: 0.9012
Epoch 6/200
32/32 [==============================] - 13s 398ms/step - loss: 0.3720 - acc: 0.9035
Epoch 7/200
32/32 [==============================] - 13s 395ms/step - loss: 0.3498 - acc: 0.9085
Epoch 8/200
32/32 [==============================] - 13s 395ms/step - loss: 0.3280 - acc: 0.9132
Epoch 9/200
32/32 [==============================] - 13s 394ms/step - loss: 0.3115 - acc: 0.9161
Epoch 10/200
32/32 [==============================] - 13s 392ms/step - loss: 0.2989 - acc: 0.9176
Epoch 11/200
32/32 [=========

KeyboardInterrupt: ignored

In [ ]:
start_words = ["empty", "boring", "happy", "bridge"  "you", "where", "when", "take", "us", "want", "remember", "friendship", "coming", "expected", "make", "do", "all", "birthday", "chance"]
temperature_rates = [1, 1.2, 1.4, 1.6]
for word in start_words:
  for temperature in temperature_rates:
    print("[START WORD]: ", word, " [TEMPERATURE]:", temperature)
    generate_soft_for_symbol(symbol_lstm, 20, [word], temperature=temperature)
    print()
    print()

[START WORD]:  empty  [TEMPERATURE]: 1
........... HAluthet

[START WORD]:  empty  [TEMPERATURE]: 1.2
..........MOfrevesth

[START WORD]:  empty  [TEMPERATURE]: 1.4
. l Yousthemecadyonv

[START WORD]:  empty  [TEMPERATURE]: 1.6
......balthinvanvest

[START WORD]:  boring  [TEMPERATURE]: 1
....................

[START WORD]:  boring  [TEMPERATURE]: 1.2
sthevadyonvestheJure

[START WORD]:  boring  [TEMPERATURE]: 1.4
sthecanedofreveuth-T

[START WORD]:  boring  [TEMPERATURE]: 1.6
sth, Onvesthecanvany

[START WORD]:  happy  [TEMPERATURE]: 1
............ So lf B

[START WORD]:  happy  [TEMPERATURE]: 1.2
 A Nofrecasthesthanv

[START WORD]:  happy  [TEMPERATURE]: 1.4
 Buzek Ahanvem6.. Co

[START WORD]:  happy  [TEMPERATURE]: 1.6
? lthexquzerecaltThe

[START WORD]:  bridgeyou  [TEMPERATURE]: 1
s Sonveconveth-(Bume

[START WORD]:  bridgeyou  [TEMPERATURE]: 1.2
pencal'mestrevetstha

[START WORD]:  bridgeyou  [TEMPERATURE]: 1.4
d julfifo AltwistAve

[START WORD]:  bridgeyou  [TEMPERATURE]: 1.6
zY

Многослойная LSTM

In [ ]:
symbol_multilayer_lstm = tf.keras.models.Sequential([
    tf.keras.layers.Masking(input_shape=(maxlen,vocab_size)),
    tf.keras.layers.LSTM(128,return_sequences=True,input_shape=(maxlen,vocab_size)),
    tf.keras.layers.LSTM(128,return_sequences=True,input_shape=(maxlen,vocab_size)),
    tf.keras.layers.LSTM(128,return_sequences=True,input_shape=(maxlen,vocab_size)),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size,activation='softmax'))
])
symbol_multilayer_lstm.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_2 (Masking)         (None, 312, 83)           0         
                                                                 
 lstm_1 (LSTM)               (None, 312, 128)          108544    
                                                                 
 lstm_2 (LSTM)               (None, 312, 128)          131584    
                                                                 
 lstm_3 (LSTM)               (None, 312, 128)          131584    
                                                                 
 time_distributed_2 (TimeDis  (None, 312, 83)          10707     
 tributed)                                                       
                                                                 
Total params: 382,419
Trainable params: 382,419
Non-trainable params: 0
________________________________________________

In [ ]:
symbol_multilayer_lstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='categorical_crossentropy',metrics=['acc'])

symbol_multilayer_lstm.fit(x1, y1, epochs=200)

Epoch 1/200
17/32 [==============>...............] - ETA: 20s - loss: 0.0254 - acc: 0.9921

KeyboardInterrupt: ignored

In [ ]:
start_words = ["empty", "boring", "happy", "bridge",  "you", "where", "when", "take", "us", "want", "remember", "friendship", "coming", "expected", "make", "do", "all", "birthday", "chance"]
temperature_rates = [1, 1.2, 1.4, 1.6]
for word in start_words:
  for temperature in temperature_rates:
    print("[START WORD]: ", word, " [TEMPERATURE]:", temperature)
    generate_soft_for_symbol(symbol_multilayer_lstm, 20, [word], temperature=temperature)
    print()
    print()

[START WORD]:  empty  [TEMPERATURE]: 1
 Ex; Altweveva Whano

[START WORD]:  empty  [TEMPERATURE]: 1.2
 B-Hevanowivevevl WO

[START WORD]:  empty  [TEMPERATURE]: 1.4
 Nevlfreveva Tutw We

[START WORD]:  empty  [TEMPERATURE]: 1.6
  Arevotwanofrevre W

[START WORD]:  boring  [TEMPERATURE]: 1
! WOha WOhanofoltwis

[START WORD]:  boring  [TEMPERATURE]: 1.2
 Cofofofofofofol'LOh

[START WORD]:  boring  [TEMPERATURE]: 1.4
 Whe-Jufok2.WOhevano

[START WORD]:  boring  [TEMPERATURE]: 1.6
 Yofofofivevanofoutw

[START WORD]:  happy  [TEMPERATURE]: 1
nofrevevevevltweveve

[START WORD]:  happy  [TEMPERATURE]: 1.2
 Whaltutwlfofofetwiv

[START WORD]:  happy  [TEMPERATURE]: 1.4
ivev SLofowivavltw C

[START WORD]:  happy  [TEMPERATURE]: 1.6
lfotanoltwiveveStwif

[START WORD]:  bridge  [TEMPERATURE]: 1
danofrevltwive'vivev

[START WORD]:  bridge  [TEMPERATURE]: 1.2
da Nofreveveveva ltw

[START WORD]:  bridge  [TEMPERATURE]: 1.4
s Yofreva Anofoca St

[START WORD]:  bridge  [TEMPERATURE]: 1.6
nl Ta Any.peva

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.10/dist-packages/keras/backend.py", line 5117, in <genexpr>
    output_ta_t = tuple(  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


veveve'v Yevlfo

[START WORD]:  friendship  [TEMPERATURE]: 1.4
.....wivava Ogevevev

[START WORD]:  friendship  [TEMPERATURE]: 1.6
,eOfopivivev WOhanof

[START WORD]:  coming  [TEMPERATURE]: 1
 Tha Ohanofofohanofo

[START WORD]:  coming  [TEMPERATURE]: 1.2
 YofreveveveOkbofrev

[START WORD]:  coming  [TEMPERATURE]: 1.4
 CHofofofofrevlt lta

[START WORD]:  coming  [TEMPERATURE]: 1.6
 BOnlfofreveve'stwiv

[START WORD]:  expected  [TEMPERATURE]: 1
 Cofofotwiveva Wheve

[START WORD]:  expected  [TEMPERATURE]: 1.2
,evanofre'vevlfrevan

[START WORD]:  expected  [TEMPERATURE]: 1.4
 Arevanofofo-xPofofr

[START WORD]:  expected  [TEMPERATURE]: 1.6
 Oh- Al YofotOha Wha

[START WORD]:  make  [TEMPERATURE]: 1
,eva Al Arevevevevev

[START WORD]:  make  [TEMPERATURE]: 1.2
,eva Jutwa Ooforeva 

[START WORD]:  make  [TEMPERATURE]: 1.4
,evevetwival'vltweve

[START WORD]:  make  [TEMPERATURE]: 1.6
,et Wha Nevevlwre're

[START WORD]:  do  [TEMPERATURE]: 1
utwiveveveRevltwever

[START WORD]:  do  [TEMPERA

Двунаправленная LSTM

In [ ]:
bidirectional_lstm = tf.keras.models.Sequential([
    tf.keras.layers.Masking(input_shape=(maxlen,vocab_size)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True,input_shape=(maxlen,vocab_size))),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True,input_shape=(maxlen,vocab_size))),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True,input_shape=(maxlen,vocab_size))),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size,activation='softmax'))
])
bidirectional_lstm.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_4 (Masking)         (None, 312, 83)           0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 312, 256)         217088    
 nal)                                                            
                                                                 
 bidirectional_4 (Bidirectio  (None, 312, 256)         394240    
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 312, 256)         394240    
 nal)                                                            
                                                                 
 time_distributed_4 (TimeDis  (None, 312, 83)          21331     
 tributed)                                            

In [ ]:
bidirectional_lstm.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy',metrics=['acc'])

bidirectional_lstm.fit(x1, y1, epochs=200)

Epoch 1/200
32/32 [==============================] - 118s 3s/step - loss: 0.9291 - acc: 0.8593
Epoch 2/200
32/32 [==============================] - 94s 3s/step - loss: 0.4257 - acc: 0.9006
Epoch 3/200
32/32 [==============================] - 94s 3s/step - loss: 0.3953 - acc: 0.9024
Epoch 4/200
32/32 [==============================] - 93s 3s/step - loss: 0.3778 - acc: 0.9040
Epoch 5/200
32/32 [==============================] - 94s 3s/step - loss: 0.3627 - acc: 0.9055
Epoch 6/200
32/32 [==============================] - 94s 3s/step - loss: 0.3522 - acc: 0.9066
Epoch 7/200
32/32 [==============================] - 94s 3s/step - loss: 0.3427 - acc: 0.9071
Epoch 8/200
32/32 [==============================] - 93s 3s/step - loss: 0.3298 - acc: 0.9087
Epoch 9/200
32/32 [==============================] - 94s 3s/step - loss: 0.3105 - acc: 0.9119
Epoch 10/200
32/32 [==============================] - 93s 3s/step - loss: 0.2796 - acc: 0.9206
Epoch 11/200
32/32 [==============================] - 93s 

KeyboardInterrupt: ignored

In [ ]:
start_words = ["empty", "happy", "bridge", "when", "take", "want", "remember", "expected", "make", "birthday", "chance"]
temperature_rates = [1, 1.2, 1.4, 1.6]
for word in start_words:
  for temperature in temperature_rates:
    print("[START WORD]: ", word, " [TEMPERATURE]:", temperature)
    generate_soft_for_symbol(symbol_multilayer_lstm, 20, [word], temperature=temperature)
    print()
    print()

[START WORD]:  empty  [TEMPERATURE]: 1
 Whanofrevevanofreva

[START WORD]:  empty  [TEMPERATURE]: 1.2
 Tha Alfofrevivevano

[START WORD]:  empty  [TEMPERATURE]: 1.4
 lfre'vevevavevere'v

[START WORD]:  empty  [TEMPERATURE]: 1.6
 TOw Whenofrevevev C

[START WORD]:  happy  [TEMPERATURE]: 1
 Nevanofofrevevevano

[START WORD]:  happy  [TEMPERATURE]: 1.2
 Butwa Nofrevetwevev

[START WORD]:  happy  [TEMPERATURE]: 1.4
'v TevltwiveveDolfre

[START WORD]:  happy  [TEMPERATURE]: 1.6
iveMoxpofreDReve:an 

[START WORD]:  bridge  [TEMPERATURE]: 1
danotwivanyutwivevav

[START WORD]:  bridge  [TEMPERATURE]: 1.2
ltwivev Weva ROfofre

[START WORD]:  bridge  [TEMPERATURE]: 1.4
danonofrevlt BUhanof

[START WORD]:  bridge  [TEMPERATURE]: 1.6
notwevevltofrevanowe

[START WORD]:  when  [TEMPERATURE]: 1
twivanofofrevevanofr

[START WORD]:  when  [TEMPERATURE]: 1.2
 Avev Wha Nofreveva 

[START WORD]:  when  [TEMPERATURE]: 1.4
,egevltwivifretwivev

[START WORD]:  when  [TEMPERATURE]: 1.6
hevevofreveOnofofo-(

